# Tarea 5 - Árbol de Desición

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [1]:
import pandas as pd
import numpy as np

## Actividad - Taller

1. Cree una clase nodo con atributos necesarios para un árbol de decisión: feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha

In [ ]:
class Nodo:
    # :param feature: Índice de la característica utilizada para la división en este nodo.
    # :param umbral: Valor del umbral utilizado para la división.
    # :param gini: Índice de Gini de este nodo.
    # :param cantidad_muestras: Número de muestras en este nodo.
    # :param valor: Valor de la clase para nodos hoja (puede ser el valor promedio o la clase más común).
    # :param izquierda: Nodo hijo izquierdo.
    # :param derecha: Nodo hijo derecho.
    def __init__(self, feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha):
        self.feature = feature
        self.umbral = umbral
        self.gini = gini
        self.cantidad_muestras = cantidad_muestras
        self.valor = valor
        self.izquierda = izquierda
        self.derecha = derecha

2. Crea una clase que implementa un árbol de decisión, utilice las funciones presentadas en clase, además incluya los siguientes hyperparámetros:
- max_depth: Cantidad máxima de variables que se pueden explorar
- min_split_samples: Cantidad mínima de muestras que deberá tener un nodo para poder ser dividido
- criterio: función que se utilizará para calcular la impuridad.

In [ ]:
class ArbolDecision:
    # :param max_depth: Profundidad máxima del árbol.
    # :param min_split_samples: Número mínimo de muestras necesarias para dividir un nodo.
    # :param criterio: Criterio utilizado para calcular la impureza ('gini' o 'entropia').
    def __init__(self, max_depth, min_split_samples=2, criterio="gini"):
        self.max_depth = max_depth
        self.min_split_samples = min_split_samples
        self.criterio = criterio
        self.raiz = None
    def _gini(self, y):
        clases, counts = np.unique(y, return_counts=True)
        p = counts / len(y)
        return 1 - np.sum(p ** 2)